# Sequence generation

- all available historical data
- include static features

- attention mechanism

- validation on DataLoch

- model explanation

# LSTM Model using Clinical + Therapy sequence

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional, Input, Reshape, BatchNormalization, Flatten, concatenate, Add, Multiply, LeakyReLU
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC, SensitivityAtSpecificity
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop, Adamax, SGD
from tensorflow.keras.initializers import Constant
from tensorflow.keras.regularizers import L1L2, L1, L2
from livelossplot import PlotLossesKeras

# from tensorflow.keras.backend.tensorflow_backend import set_session
from tensorflow.python.keras import backend as K

#internal validation
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, balanced_accuracy_score, matthews_corrcoef, auc, average_precision_score, roc_auc_score, balanced_accuracy_score, roc_curve, accuracy_score

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import pickle

# fix random seed for reproducibility
tf.random.set_seed(1234)

target_outcome = '12months'
max_codes = 100

In [ ]:
sets =  pickle.load(open('../SeqModel/sets_1year.sav', 'rb'))
sets_eval = pickle.load(open('../SeqModel/sets_eval_1year.sav', 'rb'))
code2idx = pickle.load(open('../SeqModel/all_vocab_1year.sav', 'rb'))
month2idx = pickle.load(open('../SeqModel/all_vocab_month.sav', 'rb'))
vocab_size = len(code2idx)+1
month_size = len(month2idx)+1
print(vocab_size)
print(month_size)

In [ ]:
Xt_train, Xt_val, Xt_eval, Xs_train, Xs_val, Xs_eval, Xm_train, Xm_val, Xm_eval, y_train, y_val, y_eval = sets
Xt_test, Xt_testWales, Xt_testScotland, Xs_test, Xs_testWales, Xs_testScotland, Xm_test, Xm_testWales, Xm_testScotland, y_test, y_testWales, y_testScotland = sets_eval

In [ ]:
print('Train: ', Xs_train.shape)
print('Val: ', Xs_val.shape[0])
print('Eval (internal validation): ', Xs_eval.shape[0])
print('Test: ', Xs_test.shape[0])
print('Test - Wales: ', Xs_testWales.shape[0])
print('Test - Scotland: ', Xs_testScotland.shape[0])

In [ ]:
from sklearn.utils import class_weight
sklearn_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight = dict(enumerate(sklearn_weights))

In [ ]:
# pickle.dump(code2idx_all_big, open('../SeqModel/code2idx_all_big.sav', 'wb'))
# pickle.dump(idx2code_all_big, open('../SeqModel/idx2code_all_big.sav', 'wb'))
# pickle.dump(data, open('../SeqModel/data_all_big.sav', 'wb'))

In [ ]:
target_outcome = '12months'
max_codes = 70
tab_feature_size = Xt_train.shape[1]
lr = 1e-5
clipvalue = 0.2
epoch = 1000
batch_size = 256
embedding_vector_length = 50
month_embedding_vector_length = 5
# embedding_vector_length = int(np.sqrt(vocab_size))
# embedding_vector_length = int(np.cbrt(vocab_size))
print(embedding_vector_length)

In [ ]:
def earlyFussion():
       
    inputs1 = Input(shape=tab_feature_size)
    inputs2 = Input(shape=max_codes)
    inputs3 = Input(shape=max_codes)
    
    
    #clinical embedding for lstm
    embedding = Embedding(vocab_size, 50, input_length=max_codes)(inputs2)
    
    #month embedding for lstm
    embedding_month = Embedding(month_size, 7, input_length=max_codes)(inputs3)
    
    nn = Dense(32, activation=LeakyReLU(), kernel_initializer='glorot_uniform', kernel_regularizer=L1L2(l1=0.0, l2=0.1))(inputs1)
    nn = Dropout(0.5)(nn)
    lstmClinical = Bidirectional(LSTM(units=16, return_sequences=True, kernel_regularizer=L1L2(l1=0.0, l2=0.1)))(embedding)
    lstmMonth = Bidirectional(LSTM(units=16, return_sequences=True, kernel_regularizer=L1L2(l1=0.0, l2=0.1)))(embedding_month)
    # lstm = Add()([lstmClinical, lstmMonth])
    lstm = lstmClinical
    
    # nn = Reshape((1, 32))(nn)
    # add = concatenate([nn, lstm], axis=1)
    nn = Dense(16, activation=LeakyReLU(), kernel_initializer='glorot_uniform', kernel_regularizer=L1L2(l1=0.0, l2=0.1))(nn)
    nn = Dropout(0.5)(nn)
    lstm = Bidirectional(LSTM(units=8, return_sequences=True, kernel_regularizer=L1L2(l1=0.0, l2=0.1)))(lstm)
    lstm = Dropout(0.5)(lstm)
    
    # # nn = Reshape((301, 64))(nn)
    # add = concatenate([nn, lstm], axis=1)
    nn = Dense(16, activation=LeakyReLU(), kernel_initializer='glorot_uniform', kernel_regularizer=L1L2(l1=0.0, l2=0.1))(nn)
    nn = Dropout(0.5)(nn)
    lstm = Bidirectional(LSTM(units=8, return_sequences=True, kernel_regularizer=L1L2(l1=0.0, l2=0.1)))(lstm)
    lstm = Dropout(0.5)(lstm)
    
    nn = Reshape((1, 16))(nn)
    model_tot = concatenate([nn, lstm], axis=1)
    # model_tot = BatchNormalization()(model_tot)

    model_tot = Dense(units=8, activation=LeakyReLU())(model_tot)
    nn = Dropout(0.5)(nn)
    
    model_tot = Flatten()(model_tot)
    output = Dense(1, activation='sigmoid')(model_tot)
    
    opt = RMSprop(learning_rate=1e-4, clipvalue=.5)
    metrics = [
        AUC(num_thresholds=1000, name='auc', curve='ROC'),
        AUC(num_thresholds=1000, name='auprc', curve='PR'),
        tf.keras.metrics.Precision(name='prec'),
        tf.keras.metrics.Recall(name='rec'),
        tf.keras.metrics.TrueNegatives(name='TN'),
        tf.keras.metrics.TruePositives(name='TP'),
        tf.keras.metrics.PrecisionAtRecall(0.8)
    ]
    
    loss = tf.keras.losses.BinaryCrossentropy()
    
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=output)
    model.compile(
        loss='binary_crossentropy', 
        optimizer=opt, 
        metrics=metrics)
    return model

In [ ]:
#visualise model
model = earlyFussion()
# model = earlyFussion()
print(model.summary())



In [ ]:
%%time
# TF_GPU_ALLOCATOR=cuda_malloc_async
#training
with tf.device('/GPU:0'):
    earlyStopping = EarlyStopping(monitor='val_auc', patience=50, verbose=0, mode='max', restore_best_weights=True)
    mcp_save = ModelCheckpoint('../SeqModel/seqModel_therapy_tabSeq.mdl_wts.hdf5', save_best_only=True, monitor='val_auc', mode='min')
    history = model.fit([Xt_train, Xs_train[:,:max_codes], Xm_train[:,:max_codes]], y_train, validation_data=([Xt_val, Xs_val[:,:max_codes], Xm_val[:,:max_codes]], y_val), 
                            epochs=epoch, batch_size=256, 
                        class_weight = class_weight, 
                        callbacks = [earlyStopping, mcp_save])

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
# plt.ylim(0.55,1)
plt.title('model AUC')
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
# plt.ylim(0.1, 1.15)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
%%time

# create the model
# embedding_vector_length = int(np.sqrt(vocab_size))
embedding_vector_length = 100
earlyStopping = EarlyStopping(monitor='val_auc', patience=10, verbose=0, mode='max', restore_best_weights=True)
mcp_save = ModelCheckpoint('../SeqModel/seqModel_all_new.mdl_wts.hdf5', save_best_only=True, monitor='val_auc', mode='min')
# class_weight = {0: 1, 1: 6}


with tf.device('/GPU:0'):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_codes))
    model.add(LSTM(128, return_sequences=True,
                   bias_regularizer=L1L2(l1=0.0, l2=0.01)))
    model.add(Dropout(0.5))
    model.add(LSTM(128, return_sequences=True,
                   bias_regularizer=L1L2(l1=0.0, l2=0.01)))
    model.add(Dropout(0.5))
    model.add(LSTM(128, bias_regularizer=L1L2(l1=0.0, l2=0.01)))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    # model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = RMSprop(learning_rate=5e-4, clipvalue=0.5)
    metrics = [
        AUC(num_thresholds=1000, name='auc', curve='ROC'),
        AUC(num_thresholds=1000, name='auprc', curve='PR'),
        tf.keras.metrics.Precision(name='prec'),
        tf.keras.metrics.Recall(name='rec'),
        tf.keras.metrics.TrueNegatives(name='TN'),
        tf.keras.metrics.TruePositives(name='TP'),
        tf.keras.metrics.PrecisionAtRecall(0.8)
    ]
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=metrics, )
    print(model.summary())
    history = model.fit(Xs_train[:,:max_codes], y_train, validation_data=(Xs_val[:,:max_codes], y_val), 
                        epochs=1000, batch_size=128, 
                        class_weight = class_weight, 
                        callbacks = [earlyStopping, mcp_save])



In [ ]:
model.summary()

In [ ]:
import io
out_v = io.open('../SeqModel/vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('../SeqModel/metadata.tsv', 'w', encoding='utf-8')
for index, word in enumerate(vocab):
  if (index == 0)|(index == 1)|(index == 2):
    continue  # skip 0,1,2, it's padding, satr and end
  vec = embeddings_vector[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")

out_v.close()
out_m.close()

In [ ]:
with tf.device('/CPU:0'):
    # model.evaluate(X_train, y_train)
    model.evaluate(X_val, y_val, batch_size=300)
    model.evaluate(X_eval, y_eval, batch_size=300)
    model.evaluate(X_test, y_test, batch_size=300)
    model.evaluate(X_testWales, y_testWales, batch_size=300)
    model.evaluate(X_testScotland, y_testScotland, batch_size=300)

In [ ]:
#Model evaluation function
def summariseResult (testY, preds):
    tn, fp, fn, tp = confusion_matrix(testY, preds).ravel()
    specificity = tn / (tn+fp)
    sensitivity = tp / (tp+fn)
    ppv = 100*tp/(tp+fp)
    npv = 100*tn/(fn+tn)
    acc = accuracy_score(testY, preds)
    f1score = f1_score(testY, preds, average = 'binary')
    balanceacc = balanced_accuracy_score(testY, preds)
    fpr, tpr, thresholds = roc_curve(testY, preds, pos_label=1)
    aucscore = auc(fpr, tpr)
    # aucscore = roc_auc_score(testY, preds)
    auprc = average_precision_score(testY, preds)
    # plot_confusion_matrix(model, testX, testY, cmap='viridis')  
    return np.round(acc,4), np.round(specificity,4), np.round(sensitivity,4), np.round(aucscore,4), np.round(auprc,4), np.round(balanceacc,4), np.round(f1score,4), np.round(ppv,4), np.round(npv,4)

data_test_Xs = [X_eval, X_test, X_testWales, X_testScotland]
data_test_ys = [y_eval, y_test, y_testWales, y_testScotland]
for data_test_X, data_test_y in zip(data_test_Xs, data_test_ys):
    with tf.device('/CPU:0'):
        preds = model.predict(data_test_X)
    preds = [0 if pred <0.5 else 1 for pred in preds]
    print(summariseResult(data_test_y, np.squeeze(preds)))

In [ ]:
# model.save('../SeqModel/model_all.h5')

In [ ]:
extractor = tf.keras.Model(inputs=model.inputs, outputs=model.get_layer('embedding').output)
embeddings_vector = model.layers[0].get_weights()[0]

# `embeddings` has a shape of (num_vocab, embedding_dim) 

# `word_to_index` is a mapping (i.e. dict) from words to their index, e.g. `love`: 69
words_embeddings = {w:embeddings_vector[idx] for w, idx in code2idx.items()}

vocab = list(code2idx.keys())

In [ ]:
from sklearn.decomposition import PCA
pca_2dim = PCA(random_state=0).fit_transform(embeddings_vector)[:,:2]
pca_2dim = pd.DataFrame(pca_2dim, columns=['PC1', 'PC2'])
pca_2dim['code'] = vocab

In [ ]:
asthma_readcodes = pd.read_csv('../RCodes/ClinicalCodes/definite_asthma_read_codes.csv')[['readcode_new']]
ethnicity = pd.read_csv('../RCodes/ClinicalCodes/Ethinicity.csv')[['Read.Code']]
smoking = pd.read_csv('../RCodes/ClinicalCodes/SmokingReadcodes.csv')[['Read Code']]
ICS = pd.read_csv('../RCodes/ClinicalCodes/ICS_Asthma.csv')[['code_id']]

In [ ]:
asthma_readcodes.columns = ['ReadCode']
asthma_readcodes['label'] = 'asthma specific'
asthma_readcodes['label_num'] = 0
ethnicity.columns = ['ReadCode']
ethnicity['label'] = 'ethnicity'
ethnicity['label_num'] = '1'
smoking.columns = ['ReadCode']
smoking['label'] = 'smoking'
smoking['label_num'] = '2'
ICS.columns = ['ReadCode']
ICS['label'] = 'ICS'
ICS['label_num'] = '3'

In [ ]:
df = pd.concat([asthma_readcodes, ethnicity, smoking, ICS])

In [ ]:
pca_subset = pca_2dim[pca_2dim.code.isin(df.ReadCode.unique())]

In [ ]:
pca_subset = df.merge(pca_subset, how='right', left_on='ReadCode', right_on='code')

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(pca_subset.PC1.values,pca_subset.PC2.values, c=pca_subset.label_num.values)
plt.show()
